# Casos confirmados - gráficos
> Gráficos de acumulado de infectados confirmados.

- toc: false 
- badges: false
- comments: false

Última actualización 29/02/2020 - 23:39

## Total

In [1]:
#collapse

import requests
import pandas as pd
from io import BytesIO
import numpy as np
import altair as alt

# get data
response = requests.get('https://docs.google.com/spreadsheet/ccc?key=1FwOXbkWeuS1LHrKxhSfepGtsgQ6iIEQZmWig_JKo8z0&output=csv')
assert response.status_code == 200, 'Wrong status code'
data = response.content
df = pd.read_csv(BytesIO(data), parse_dates=['Fecha'], dayfirst=True)

# Create a selection that chooses the nearest point & selects based on x-value
source = df
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Total confirmados'], empty='none')

# The basic line
line = alt.Chart(source).mark_line(interpolate='basis').encode(
    x='monthdate(Fecha):O',
    y='Total confirmados:Q',
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(source).mark_point().encode(
    x='monthdate(Fecha):O',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Total confirmados:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(source).mark_rule(color='gray').encode(
    x='monthdate(Fecha):O',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=800, height=400
)

alt.LayerChart(...)

## Por comunidad autónoma

Haz click en la leyenda para activar una única línea.

In [2]:
#collapse

import requests
import pandas as pd
from io import BytesIO
import numpy as np
import altair as alt

# data download
response = requests.get('https://docs.google.com/spreadsheet/ccc?key=1FwOXbkWeuS1LHrKxhSfepGtsgQ6iIEQZmWig_JKo8z0&output=csv')
assert response.status_code == 200, 'Wrong status code'
data = response.content
df = pd.read_csv(BytesIO(data), parse_dates=['Fecha'], dayfirst=True)

# plot
cols = list(df.columns)
cols.remove('Total confirmados')
cols.remove('Fecha')
df = df.set_index('Fecha')[cols]
source = df.reset_index().melt('Fecha', var_name='category', value_name='Confirmados')
# plot
selection = alt.selection_multi(fields=['category'], bind='legend')

alt.Chart(source).mark_line(interpolate='basis').encode(
    alt.X('monthdate(Fecha):O', axis=alt.Axis()),
    alt.Y('Confirmados:Q'),
    alt.Color('category:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).properties(
    width=800, height=400
).add_selection(
    selection
)

alt.Chart(...)

## Comparación con otros países

Haz click en la leyenda para activar una única línea.

Datos tomados de fuentes externas. Ver sección "Sobre esta herramienta" para más información.

In [35]:
# collapse

# get data
response = requests.get('https://docs.google.com/spreadsheet/ccc?key=1Z7VQ5xlf3BaTx_LBBblsW4hLoGYWnZyog3jqsS9Dbgc&output=csv&gid=0')
assert response.status_code == 200, 'Wrong status code'
data = response.content
df = pd.read_csv(BytesIO(data))
df = df[df.columns[2:]][1:65].T.reset_index(drop=True)
col_names = df.iloc[0]
col_names[1] = 'Fecha'
df.columns = col_names
df = df.drop(df.index[0])
df['Fecha'] = pd.to_datetime(df['Fecha'], utc=True)
df = df.set_index('Fecha')
df = df.applymap(lambda a: str(a).replace(',',''))
df = df.apply(pd.to_numeric, errors='coerce')

# plot
keep = ['Spain', 'Italy', 'South Korea', 'Iran', 'Japan', 'France', 'Singapore']
df = df[keep]
# the last day is dicarded until the day is (almost) over
plus_all = 48
df = df[0:plus_all]

source = df.reset_index().melt('Fecha', var_name='category', value_name='Confirmados')
selection = alt.selection_multi(fields=['category'], bind='legend')

alt.Chart(source).mark_line(interpolate='basis').encode(
    alt.X('monthdate(Fecha):O', axis=alt.Axis()),
    alt.Y('Confirmados:Q'),
    alt.Color('category:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).properties(
    width=800, height=400
).add_selection(
    selection
)

alt.Chart(...)

## Comparación con otros países alineando las curvas temporalmente

Tomamos como inicio de las curvas aproximadamente el último día que hay 3 casos.

Datos tomados de fuentes externas. Ver sección "Sobre esta herramienta" para más información.

Haz click en la leyenda para activar una única línea.

### a 5 días

In [41]:
# collapse
# get data
response = requests.get('https://docs.google.com/spreadsheet/ccc?key=1Z7VQ5xlf3BaTx_LBBblsW4hLoGYWnZyog3jqsS9Dbgc&output=csv&gid=0')
assert response.status_code == 200, 'Wrong status code'
data = response.content
df = pd.read_csv(BytesIO(data))
df = df[df.columns[2:]][1:65].T.reset_index(drop=True)
col_names = df.iloc[0]
col_names[1] = 'Fecha'
df.columns = col_names
df = df.drop(df.index[0])
df['Fecha'] = pd.to_datetime(df['Fecha'], utc=True)
df = df.set_index('Fecha')
df = df.applymap(lambda a: str(a).replace(',',''))
df = df.apply(pd.to_numeric, errors='coerce')
keep = ['Spain', 'Italy', 'South Korea', 'Iran', 'Japan', 'France', 'Singapore']
df = df[keep]

# align data
aligned_data = {}
plus_days = 5
aligned_data['Spain'] = np.array(df['Spain'][42:42+plus_days])
aligned_data['Italy'] = np.array(df['Italy'][37:37+plus_days])
aligned_data['South Korea'] = np.array(df['South Korea'][16:16+plus_days])
aligned_data['Iran'] = np.array(df['Iran'][37:37+plus_days])
aligned_data['Japan'] = np.array(df['Japan'][14:14+plus_days])
aligned_data['France'] = np.array(df['France'][14:14+plus_days])
aligned_data['Singapore'] = np.array(df['Singapore'][12:12+plus_days])
df_aligned = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in aligned_data.items() ]))
df = df_aligned
df

,Spain,Italy,South Korea,Iran,Japan,France,Singapore
0,3.0,3.0,4.0,2.0,3.0,3.0,4.0
1,9.0,3.0,6.0,5.0,7.0,4.0,4.0
2,13.0,20.0,11.0,18.0,11.0,5.0,5.0
3,25.0,79.0,12.0,28.0,14.0,6.0,7.0
4,37.0,152.0,15.0,43.0,17.0,6.0,10.0


In [18]:
# collapse
# plot
source = df.reset_index().melt('index', var_name='category', value_name='Confirmados')
source.index = list(source.index)
source.index = source.index.astype(int)
selection = alt.selection_multi(fields=['category'], bind='legend')

alt.Chart(source).mark_line(interpolate='basis').encode(
    alt.X('index:Q', axis=alt.Axis(tickRound=True), title='Número de días'),
    alt.Y('Confirmados:Q'),
    alt.Color('category:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).properties(
    width=800, height=400
).add_selection(
    selection
)

alt.Chart(...)

### a 10 días

In [43]:
# collapse
# get data
response = requests.get('https://docs.google.com/spreadsheet/ccc?key=1Z7VQ5xlf3BaTx_LBBblsW4hLoGYWnZyog3jqsS9Dbgc&output=csv&gid=0')
assert response.status_code == 200, 'Wrong status code'
data = response.content
df = pd.read_csv(BytesIO(data))
df = df[df.columns[2:]][1:65].T.reset_index(drop=True)
col_names = df.iloc[0]
col_names[1] = 'Fecha'
df.columns = col_names
df = df.drop(df.index[0])
df['Fecha'] = pd.to_datetime(df['Fecha'], utc=True)
df = df.set_index('Fecha')
df = df.applymap(lambda a: str(a).replace(',',''))
df = df.apply(pd.to_numeric, errors='coerce')
keep = ['Spain', 'Italy', 'South Korea', 'Iran', 'Japan', 'France', 'Singapore']
df = df[keep]

# align data
aligned_data = {}
plus_days = 10
plus_spain = 6
aligned_data['Spain'] = np.array(df['Spain'][42:42+plus_spain])
aligned_data['Italy'] = np.array(df['Italy'][37:37+plus_days])
aligned_data['South Korea'] = np.array(df['South Korea'][16:16+plus_days])
aligned_data['Iran'] = np.array(df['Iran'][37:37+plus_days])
aligned_data['Japan'] = np.array(df['Japan'][14:14+plus_days])
aligned_data['France'] = np.array(df['France'][14:14+plus_days])
aligned_data['Singapore'] = np.array(df['Singapore'][12:12+plus_days])
df_aligned = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in aligned_data.items() ]))
df = df_aligned
df

,Spain,Italy,South Korea,Iran,Japan,France,Singapore
0,3.0,3.0,4.0,2.0,3.0,3.0,4.0
1,9.0,3.0,6.0,5.0,7.0,4.0,4.0
2,13.0,20.0,11.0,18.0,11.0,5.0,5.0
3,25.0,79.0,12.0,28.0,14.0,6.0,7.0
4,37.0,152.0,15.0,43.0,17.0,6.0,10.0
5,58.0,228.0,15.0,61.0,20.0,6.0,13.0
6,NaN,323.0,16.0,95.0,20.0,6.0,16.0
7,NaN,466.0,23.0,139.0,20.0,6.0,18.0
8,NaN,653.0,23.0,245.0,23.0,6.0,18.0
9,NaN,897.0,24.0,388.0,25.0,6.0,18.0


In [44]:
# collapse
# plot
source = df.reset_index().melt('index', var_name='category', value_name='Confirmados')
source.index = list(source.index)
source.index = source.index.astype(int)
selection = alt.selection_multi(fields=['category'], bind='legend')

alt.Chart(source).mark_line(interpolate='basis').encode(
    alt.X('index:Q', axis=alt.Axis(tickRound=True), title='Número de días'),
    alt.Y('Confirmados:Q'),
    alt.Color('category:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).properties(
    width=800, height=400
).add_selection(
    selection
)

alt.Chart(...)

### a 20 días

In [45]:
# collapse
# get data
response = requests.get('https://docs.google.com/spreadsheet/ccc?key=1Z7VQ5xlf3BaTx_LBBblsW4hLoGYWnZyog3jqsS9Dbgc&output=csv&gid=0')
assert response.status_code == 200, 'Wrong status code'
data = response.content
df = pd.read_csv(BytesIO(data))
df = df[df.columns[2:]][1:65].T.reset_index(drop=True)
col_names = df.iloc[0]
col_names[1] = 'Fecha'
df.columns = col_names
df = df.drop(df.index[0])
df['Fecha'] = pd.to_datetime(df['Fecha'], utc=True)
df = df.set_index('Fecha')
df = df.applymap(lambda a: str(a).replace(',',''))
df = df.apply(pd.to_numeric, errors='coerce')
keep = ['Spain', 'Italy', 'South Korea', 'Iran', 'Japan', 'France', 'Singapore']
df = df[keep]

# align data
aligned_data = {}
plus_days = 20
plus_spain = 6
plus_iran = 11
plus_italy = 11

aligned_data['Spain'] = np.array(df['Spain'][42:42+plus_spain])
aligned_data['Italy'] = np.array(df['Italy'][37:37+plus_italy])
aligned_data['South Korea'] = np.array(df['South Korea'][16:16+plus_days])
aligned_data['Iran'] = np.array(df['Iran'][37:37+plus_iran])
aligned_data['Japan'] = np.array(df['Japan'][14:14+plus_days])
aligned_data['France'] = np.array(df['France'][14:14+plus_days])
aligned_data['Singapore'] = np.array(df['Singapore'][12:12+plus_days])
df_aligned = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in aligned_data.items() ]))
df = df_aligned

# plot
source = df.reset_index().melt('index', var_name='category', value_name='Confirmados')
source.index = list(source.index)
source.index = source.index.astype(int)
selection = alt.selection_multi(fields=['category'], bind='legend')

alt.Chart(source).mark_line(interpolate='basis').encode(
    alt.X('index:Q', axis=alt.Axis(tickRound=True), title='Número de días'),
    alt.Y('Confirmados:Q'),
    alt.Color('category:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).properties(
    width=800, height=400
).add_selection(
    selection
)

alt.Chart(...)

### a 30 días

In [46]:
# collapse
# get data
response = requests.get('https://docs.google.com/spreadsheet/ccc?key=1Z7VQ5xlf3BaTx_LBBblsW4hLoGYWnZyog3jqsS9Dbgc&output=csv&gid=0')
assert response.status_code == 200, 'Wrong status code'
data = response.content
df = pd.read_csv(BytesIO(data))
df = df[df.columns[2:]][1:65].T.reset_index(drop=True)
col_names = df.iloc[0]
col_names[1] = 'Fecha'
df.columns = col_names
df = df.drop(df.index[0])
df['Fecha'] = pd.to_datetime(df['Fecha'], utc=True)
df = df.set_index('Fecha')
df = df.applymap(lambda a: str(a).replace(',',''))
df = df.apply(pd.to_numeric, errors='coerce')
keep = ['Spain', 'Italy', 'South Korea', 'Iran', 'Japan', 'France', 'Singapore']
df = df[keep]

# align data
aligned_data = {}
plus_days = 40
plus_spain = 6
plus_iran = 11
plus_italy = 11
plus_s_korea = 32

aligned_data['Spain'] = np.array(df['Spain'][42:42+plus_spain])
aligned_data['Italy'] = np.array(df['Italy'][37:37+plus_italy])
aligned_data['South Korea'] = np.array(df['South Korea'][16:16+plus_s_korea])
aligned_data['Iran'] = np.array(df['Iran'][37:37+plus_iran])
aligned_data['Japan'] = np.array(df['Japan'][14:14+plus_days])
aligned_data['France'] = np.array(df['France'][14:14+plus_days])
aligned_data['Singapore'] = np.array(df['Singapore'][12:12+plus_days])
df_aligned = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in aligned_data.items() ]))
df = df_aligned

# plot
source = df.reset_index().melt('index', var_name='category', value_name='Confirmados')
source.index = list(source.index)
source.index = source.index.astype(int)
selection = alt.selection_multi(fields=['category'], bind='legend')

alt.Chart(source).mark_line(interpolate='basis').encode(
    alt.X('index:Q', axis=alt.Axis(tickRound=True), title='Número de días'),
    alt.Y('Confirmados:Q'),
    alt.Color('category:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).properties(
    width=800, height=400
).add_selection(
    selection
)

alt.Chart(...)